## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [9]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

170500096/170498071 [==============================] - 20s 0us/step
(50000, 32, 32, 3)


In [3]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

D:\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [15]:

classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(32,kernel_size=(3,3),input_shape= (32,32,3), activation = 'relu'))#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization())

'''自己決定MaxPooling2D放在哪裡'''
classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(64,kernel_size=(4,4)))
classifier.add(BatchNormalization())

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(units=100, activation = 'relu')) #output_dim=100,activation=relu
               
#輸出
classifier.add(Dense(units=10, activation = 'softmax'))
               
#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=100)

Epoch 1/100
50000/50000 [==============================] - 64s 1ms/step - loss: 1.4810 - accuracy: 0.5260
Epoch 2/100
50000/50000 [==============================] - 64s 1ms/step - loss: 0.9346 - accuracy: 0.6755
Epoch 3/100
50000/50000 [==============================] - 64s 1ms/step - loss: 0.7598 - accuracy: 0.7354
Epoch 4/100
50000/50000 [==============================] - 64s 1ms/step - loss: 0.6450 - accuracy: 0.7743
Epoch 5/100
50000/50000 [==============================] - 67s 1ms/step - loss: 0.5444 - accuracy: 0.8067
Epoch 6/100
50000/50000 [==============================] - 77s 2ms/step - loss: 0.4472 - accuracy: 0.8434
Epoch 7/100
50000/50000 [==============================] - 75s 2ms/step - loss: 0.3783 - accuracy: 0.8653
Epoch 8/100
50000/50000 [==============================] - 75s 2ms/step - loss: 0.3133 - accuracy: 0.8890
Epoch 9/100
50000/50000 [==============================] - 75s 2ms/step - loss: 0.2546 - accuracy: 0.9110
Epoch 10/100
50000/50000 [====================

50000/50000 [==============================] - 65s 1ms/step - loss: 0.0238 - accuracy: 0.9924
Epoch 78/100
50000/50000 [==============================] - 65s 1ms/step - loss: 0.0287 - accuracy: 0.9911
Epoch 79/100
50000/50000 [==============================] - 64s 1ms/step - loss: 0.0232 - accuracy: 0.9928
Epoch 80/100
50000/50000 [==============================] - 65s 1ms/step - loss: 0.0197 - accuracy: 0.9937
Epoch 81/100
50000/50000 [==============================] - 64s 1ms/step - loss: 0.0264 - accuracy: 0.9917
Epoch 82/100
50000/50000 [==============================] - 65s 1ms/step - loss: 0.0273 - accuracy: 0.9918
Epoch 83/100
50000/50000 [==============================] - 64s 1ms/step - loss: 0.0255 - accuracy: 0.9917
Epoch 84/100
50000/50000 [==============================] - 65s 1ms/step - loss: 0.0211 - accuracy: 0.9931
Epoch 85/100
50000/50000 [==============================] - 67s 1ms/step - loss: 0.0195 - accuracy: 0.9934
Epoch 86/100
50000/50000 [========================

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [16]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[8.6829023e-05, 4.9923351e-06, 3.1345704e-07, 8.9464217e-01,
        1.0501981e-01, 8.9941764e-08, 2.5749476e-09, 2.4568135e-04,
        1.3331296e-08, 4.0483363e-12]], dtype=float32)